## 2.13. 프로젝트 2 : 날씨 좋은 월요일 오후 세 시, 자전거 타는 사람은 몇 명?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### (1) 데이터 가져오기

In [2]:
# bike_train = pd.read_csv('../../../data/data/bike-sharing-demand/train.csv')
# bike_test  = pd.read_csv('../../../data/data/bike-sharing-demand/test.csv')
bike_train = pd.read_csv('02/data/train.csv')
bike_test  = pd.read_csv('02/data/test.csv')

In [3]:
bike_train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [4]:
bike_test

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014
...,...,...,...,...,...,...,...,...,...
6488,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014
6489,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014
6490,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014
6491,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981


In [5]:
bike_train['season'].value_counts()

4    2734
2    2733
3    2733
1    2686
Name: season, dtype: int64

In [6]:
bike_train['holiday'].value_counts()

0    10575
1      311
Name: holiday, dtype: int64

In [7]:
bike_train['workingday'].value_counts()

1    7412
0    3474
Name: workingday, dtype: int64

In [8]:
grouped = bike_train['count'].groupby([bike_train['holiday'], bike_train['workingday']]).count()
grouped

holiday  workingday
0        0             3163
         1             7412
1        0              311
Name: count, dtype: int64

In [9]:
print(6493/311)

20.877813504823152


In [10]:
print(365/11)

33.18181818181818


 - [미국의 공휴일](https://ko.wikipedia.org/wiki/%EB%AF%B8%EA%B5%AD%EC%9D%98_%EA%B3%B5%ED%9C%B4%EC%9D%BC)은 1년에 약 11일.
 - 위의 계산과 맞지 않고, 어떻게 해석해야 할 지 잘 모르겠다.

In [11]:
bike_train['weather'].value_counts()

1    7192
2    2834
3     859
4       1
Name: weather, dtype: int64

weather
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 


### (2) `datetime` 컬럼을 `datetime` 자료형으로 변환하고 연, 월, 일, 시, 분, 초까지 6가지 컬럼 생성하기

In [12]:
a = bike_train.iloc[0,0]
a

'2011-01-01 00:00:00'

In [13]:
type(a)

str

In [14]:
aa = pd.to_datetime(a)
aa

Timestamp('2011-01-01 00:00:00')

In [15]:
type(aa)

pandas._libs.tslibs.timestamps.Timestamp

In [16]:
print(aa.year,aa.month,aa.day,aa.hour,aa.minute,aa.second)

2011 1 1 0 0 0


In [17]:
b = bike_train.iloc[1,0]
b

'2011-01-01 01:00:00'

In [18]:
type(b)

str

In [19]:
bb = pd.to_datetime(b)
bb

Timestamp('2011-01-01 01:00:00')

In [20]:
type(bb)

pandas._libs.tslibs.timestamps.Timestamp

In [21]:
print(bb.year,bb.month,bb.day,bb.hour,bb.minute,bb.second)

2011 1 1 1 0 0


In [22]:
for i in range(100):
    c = bike_train.iloc[i,0]
    cc = pd.to_datetime(c)
    print(cc.year,cc.month,cc.day,cc.hour,cc.minute,cc.second)

2011 1 1 0 0 0
2011 1 1 1 0 0
2011 1 1 2 0 0
2011 1 1 3 0 0
2011 1 1 4 0 0
2011 1 1 5 0 0
2011 1 1 6 0 0
2011 1 1 7 0 0
2011 1 1 8 0 0
2011 1 1 9 0 0
2011 1 1 10 0 0
2011 1 1 11 0 0
2011 1 1 12 0 0
2011 1 1 13 0 0
2011 1 1 14 0 0
2011 1 1 15 0 0
2011 1 1 16 0 0
2011 1 1 17 0 0
2011 1 1 18 0 0
2011 1 1 19 0 0
2011 1 1 20 0 0
2011 1 1 21 0 0
2011 1 1 22 0 0
2011 1 1 23 0 0
2011 1 2 0 0 0
2011 1 2 1 0 0
2011 1 2 2 0 0
2011 1 2 3 0 0
2011 1 2 4 0 0
2011 1 2 6 0 0
2011 1 2 7 0 0
2011 1 2 8 0 0
2011 1 2 9 0 0
2011 1 2 10 0 0
2011 1 2 11 0 0
2011 1 2 12 0 0
2011 1 2 13 0 0
2011 1 2 14 0 0
2011 1 2 15 0 0
2011 1 2 16 0 0
2011 1 2 17 0 0
2011 1 2 18 0 0
2011 1 2 19 0 0
2011 1 2 20 0 0
2011 1 2 21 0 0
2011 1 2 22 0 0
2011 1 2 23 0 0
2011 1 3 0 0 0
2011 1 3 1 0 0
2011 1 3 4 0 0
2011 1 3 5 0 0
2011 1 3 6 0 0
2011 1 3 7 0 0
2011 1 3 8 0 0
2011 1 3 9 0 0
2011 1 3 10 0 0
2011 1 3 11 0 0
2011 1 3 12 0 0
2011 1 3 13 0 0
2011 1 3 14 0 0
2011 1 3 15 0 0
2011 1 3 16 0 0
2011 1 3 17 0 0
2011 1 3 18 0 0
201

In [23]:
d = bike_train['datetime']
dd = pd.to_datetime(d)
type(dd[0])
#ddd = dd.year

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
bike_train = pd.read_csv('../../../data/data/bike-sharing-demand/train.csv')
bike_train['year']=1
bike_train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,1
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,1
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,1
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,1
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,1
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,1


In [25]:
bike_train = pd.read_csv('../../../data/data/bike-sharing-demand/train.csv')
bike_train['year']=bike_train['datetime']
bike_train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,2011-01-01 00:00:00
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,2011-01-01 01:00:00
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2011-01-01 02:00:00
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,2011-01-01 03:00:00
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,2012-12-19 19:00:00
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,2012-12-19 20:00:00
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,2012-12-19 21:00:00
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,2012-12-19 22:00:00


In [26]:
bike_train = pd.read_csv('../../../data/data/bike-sharing-demand/train.csv')
bike_train['year'] = pd.DatetimeIndex(bike_train['datetime']).year
bike_train['month'] = pd.DatetimeIndex(bike_train['datetime']).month
bike_train['day'] = pd.DatetimeIndex(bike_train['datetime']).day
bike_train['hour'] = pd.DatetimeIndex(bike_train['datetime']).hour
bike_train['minute'] = pd.DatetimeIndex(bike_train['datetime']).minute
bike_train['second'] = pd.DatetimeIndex(bike_train['datetime']).second
bike_train = bike_train[['year', 'month', 'day', 'hour',
                        'season','holiday','workingday','weather','temp',
                        'atemp','humidity','windspeed','count']]
bike_train.head()

,year,month,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011,1,1,0,1,0,0,1,9.84,14.395,81,0.0,16
1,2011,1,1,1,1,0,0,1,9.02,13.635,80,0.0,40
2,2011,1,1,2,1,0,0,1,9.02,13.635,80,0.0,32
3,2011,1,1,3,1,0,0,1,9.84,14.395,75,0.0,13
4,2011,1,1,4,1,0,0,1,9.84,14.395,75,0.0,1


In [27]:
bike_train.shape

(10886, 13)

In [28]:
bike_train['year'].value_counts()

2012    5464
2011    5422
Name: year, dtype: int64

### (3) year, month, day, hour, minute, second 데이터 개수 시각화하기

In [29]:
grouped = bike_train['count'].groupby([bike_train['year']])

In [30]:
# 각 게임별 결제 금액 시각화하기
ax = sns.barplot(data=grouped.reset_index(), x='year', y='count')

containers = ax.containers[-1]
containers0 = ax.containers[0]
ax.bar_label(containers, labels=[f'{x:,.0f}' for x in containers.datavalues], label_type='center')
ax.bar_label(containers0, labels=[f'{x:,.0f}' for x in containers0.datavalues], label_type='center')
plt.title('Payment Amount By Months and Games')
plt.show()

AttributeError: 'SeriesGroupBy' object has no attribute 'reset_index'

In [ ]:
ax = sns.countplot(data=bike_train, x='year')

containers = ax.containers[0]
ax.bar_label(containers, labels=[f'{x:,.0f}' for x in containers.datavalues], label_type='center')
containers0 = ax.containers[-1]
ax.bar_label(containers0, labels=[f'{x:,.0f}' for x in containers0.datavalues], label_type='center')
plt.title('Bike Sharing Demand by year')
plt.show()

### (4) X, y 컬럼 선택 및 train/test 데이터 분리

### (5) LinearRegression 모델 학습

### (6) 학습된 모델로 X_test에 대한 예측값 출력 및 손실함수값 계산

### (7) x축은 temp 또는 humidity로, y축은 count로 예측 결과 시각화하기